### Preparación de archivos y procesamiento de datos

In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
import string
from nltk.corpus import stopwords

In [2]:
data_items = pd.read_csv('BX_Books.csv', sep=';', error_bad_lines=False, encoding='latin-1')
data = pd.read_csv('BX-Book-Ratings.csv', sep=';', error_bad_lines=False, encoding='latin-1')

In [3]:
data.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [4]:
data_items.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton & Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [5]:
books_data = data_items.drop(['Year-Of-Publication','Image-URL-S','Image-URL-M','Image-URL-L'],axis=1)
ratings = books_data.merge(data,on='ISBN')

In [6]:
number_of_ratings = ratings.groupby('ISBN')['Book-Rating'].count().reset_index()
number_of_ratings.rename(columns={'Book-Rating':'Number of Book-Rating'}, inplace=True)
ratings = ratings.merge(number_of_ratings,on='ISBN')

In [7]:
ratings = ratings[ratings['Number of Book-Rating'] >= 30]
ratings.shape

(315065, 7)

In [8]:
ratings.drop_duplicates(['User-ID', 'Book-Title'], inplace=True)

In [9]:
ratings.shape

(313546, 7)

In [10]:
book_pivot=ratings.pivot_table(columns='User-ID',index='Book-Title',values='Book-Rating')
book_pivot.fillna(0, inplace=True)
book_pivot

User-ID,8,9,10,14,16,17,26,32,39,42,...,278831,278832,278836,278838,278843,278844,278846,278849,278851,278854
Book-Title,,,,,,,,,,,,,,,,,,,,,
'Salem's Lot,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10 Lb. Penalty,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16 Lighthouse Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1984,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zoya,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"\O\"" Is for Outlaw""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"\Surely You're Joking, Mr. Feynman!\"": Adventures of a Curious Character""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
book_sparse=csr_matrix(book_pivot)

### Modelo KNN


In [12]:
from sklearn.neighbors import NearestNeighbors
k = 10
model=NearestNeighbors(n_neighbors=k,algorithm='brute')
model.fit(book_sparse)

NearestNeighbors(algorithm='brute', n_neighbors=10)

In [13]:
distances,suggestions=model.kneighbors(book_pivot.iloc[253,:].values.reshape(1,-1))

In [14]:
distances

array([[ 0.        , 34.2636834 , 34.7706773 , 35.42597917, 35.58089375,
        35.60898763, 36.26292873, 36.31803959, 36.33180425, 36.35931793]])

In [15]:
suggestions

array([[ 253, 3541, 1180, 2622, 1181, 2845, 3196, 1369, 2333,  637]],
      dtype=int64)

In [16]:
for i in range(len(suggestions)):
    print(book_pivot.index[suggestions[i]])

Index(['Angle of Repose (Contemporary American Fiction)', 'Thornyhold',
       'Golden Cup', 'Tall, Dark, and Deadly', 'Golden Orange',
       'The Curse of the Mummy's Tomb (Goosebumps, No 5)', 'The Midnight Hour',
       'Icon', 'Second Wind', 'Commitments'],
      dtype='object', name='Book-Title')


In [17]:
def reco(book_name):
    book_id=np.where(book_pivot.index==book_name)[0][0]
    distances,suggestions=model.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1,-1))
    
    for i in range(len(suggestions)):
        if i==0:
            print("the suggestions are ",book_name,"are : ")
        if not i:
            print(book_pivot.index[suggestions[i]])

In [18]:
reco('Jurassic Park')

the suggestions are  Jurassic Park are : 
Index(['Jurassic Park', 'McNally's Caper (Archy McNally Novels (Paperback))',
       'GEMINI CONTENDERS', 'Night Mare #06', 'After Dark', 'Golden Cup',
       'Fatal Terrain', 'Night of the Living Dummy (Goosebumps, No 7)',
       'Women in His Life', 'The Satanic Verses'],
      dtype='object', name='Book-Title')


In [19]:
reco("Harry Potter and the Sorcerer's Stone (Book 1)")

the suggestions are  Harry Potter and the Sorcerer's Stone (Book 1) are : 
Index(['Harry Potter and the Sorcerer's Stone (Book 1)', 'Falling Backwards',
       'Golden Cup', 'Fatal Terrain',
       'Attack of the Mutant (Goosebumps, No 25)', 'The Invitation',
       'McNally's Caper (Archy McNally Novels (Paperback))',
       'GEMINI CONTENDERS', 'Hidden Leaves (Debeers)',
       'The Mystery of the Cupboard (Indian in the Cupboard)'],
      dtype='object', name='Book-Title')


In [20]:
reco('The Two Towers (The Lord of the Rings, Part 2)')

the suggestions are  The Two Towers (The Lord of the Rings, Part 2) are : 
Index(['The Two Towers (The Lord of the Rings, Part 2)',
       'The Return of the King (The Lord of the Rings, Part 3)', 'Golden Cup',
       'Tall, Dark, and Deadly', 'Golden Orange',
       'The Curse of the Mummy's Tomb (Goosebumps, No 5)', 'Thornyhold',
       'The Satanic Verses', 'Snagged', 'The Midnight Hour'],
      dtype='object', name='Book-Title')


In [21]:
def recoForInteface(book_name):
  book_id=np.where(book_pivot.index==book_name)[0][0]
  distances,suggestions=model.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1,-1))
  #result = ""
  #result += "the suggestions for "+book_name+" are:"
  result = []
  for i in range(len(suggestions)):
    #if i==0:
    #  result += "the suggestions for "+book_name+" are:"
    #if not i:
    result.append(book_pivot.index[suggestions[i]])
  return result[0].tolist()

In [22]:
recoForInteface('Jurassic Park')

['Jurassic Park',
 "McNally's Caper (Archy McNally Novels (Paperback))",
 'GEMINI CONTENDERS',
 'Night Mare #06',
 'After Dark',
 'Golden Cup',
 'Fatal Terrain',
 'Night of the Living Dummy (Goosebumps, No 7)',
 'Women in His Life',
 'The Satanic Verses']

In [23]:
x = recoForInteface('Jurassic Park')
type(x)
len(x)

10

In [24]:
listaDeTitulos = list(set(ratings["Book-Title"]))
listaDeTitulos

['Sick of Shadows',
 'B Is for Burglar (Kinsey Millhone Mysteries (Paperback))',
 "I Know This Much Is True (Oprah's Book Club)",
 'The Janson Directive',
 'Love You Forever',
 'The Life and Loves of a She-Devil',
 'The Te of Piglet',
 'Prince Charming',
 'Distant Shores',
 'Tuck Everlasting',
 'Kindred (Black Women Writers Series)',
 'Dilbert Fugitive From The Cubicle Police',
 'Golem in the Gears (Xanth Novels (Paperback))',
 'The Jury',
 'Voyage of the Dawn Treader',
 'Last Man Standing',
 "Stephen Coonts' Deep Black (Deep Black)",
 "Song of Solomon (Oprah's Book Club (Paperback))",
 'Jacob Have I Loved',
 'All That Remains (Kay Scarpetta Mysteries (Paperback))',
 'A Murderous Yarn (Needlecraft Mysteries)',
 'The Claiming of Sleeping Beauty (Sleeping Beauty)',
 'Dead Run',
 'Welcome to Dead House (Goosebumps, No 1)',
 'River, Cross My Heart : A Novel',
 "Timothy's Game",
 'Blackout',
 'Protein Power: The High-Protein/Low Carbohydrate Way to Lose Weight, Feel Fit, and Boost Your Heal

### Modelo TDFIF

In [25]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [26]:
books = ratings['Book-Title'].unique()
books

array(["The Kitchen God's Wife", 'The Testament',
       'Beloved (Plume Contemporary Fiction)', ...,
       'Paddy Clarke Ha Ha Ha', 'A Tangled Web',
       'The Purpose-Driven Life: What on Earth Am I Here For?'],
      dtype=object)

In [27]:
def convert_to_list(sentence):
    # Check characters to see if they are in punctuation
    nopunc = [char for char in sentence if char not in string.punctuation]
    #print(nopunc)
    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    #print(nopunc)
    # Now just remove any stopwords
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [28]:
books_word = []
for title in books:
    books_word.append(convert_to_list(title))
books_word

[['Kitchen', 'Gods', 'Wife'],
 ['Testament'],
 ['Beloved', 'Plume', 'Contemporary', 'Fiction'],
 ['New', 'Vegetarian', 'Bold', 'Beautiful', 'Recipes', 'Every', 'Occasion'],
 ['Wild', 'Animus'],
 ['Airframe'],
 ['Timeline'],
 ['Kill', 'Mockingbird'],
 ['Seabiscuit', 'American', 'Legend'],
 ['Pigs', 'Heaven'],
 ['Downtown'],
 ['Ill', 'Seeing'],
 ['Corner', 'Eye'],
 ['Isle', 'Dogs'],
 ['Purity', 'Death'],
 ['Year', 'Different', 'Stories'],
 ['Left', 'Behind', 'Novel', 'Earths', 'Last', 'Days', 'Left', 'Behind', '1'],
 ['Street', 'Lawyer'],
 ['LONESOME', 'DOVE'],
 ['Breathing', 'Lessons'],
 ['Joy', 'Luck', 'Club'],
 ['Tao', 'Pooh'],
 ['Seabiscuit'],
 ['Lifes',
  'Little',
  'Instruction',
  'Book',
  'Lifes',
  'Little',
  'Instruction',
  'Books',
  'Paperback'],
 ['Starship', 'Troopers'],
 ['Ruby', 'Smoke', 'Sally', 'Lockhart', 'Trilogy', 'Book', '1'],
 ['Catcher', 'Rye'],
 ['Midnight', 'Garden', 'Good', 'Evil', 'Savannah', 'Story'],
 ['Pretend', 'Dont', 'See'],
 ['Rich',
  'Dad',
  'Poo

In [29]:
CV_Matrix = CountVectorizer(analyzer=convert_to_list).fit_transform(books)

In [30]:
print(CV_Matrix.shape)
print(CV_Matrix.nnz)

(3859, 4563)
12643


In [31]:
TfIdf_Matrix = TfidfTransformer().fit_transform(CV_Matrix)

In [32]:
Desc_sim = linear_kernel(TfIdf_Matrix,TfIdf_Matrix)

In [33]:
df = pd.DataFrame(books,columns=['title'])
indices = pd.Series(df.index, index=df['title'].apply(lambda x: x.lower()))
indices.head(3)

title
the kitchen god's wife                  0
the testament                           1
beloved (plume contemporary fiction)    2
dtype: int64

In [34]:
def book_recomendation(title):
    title = title.lower()
    idx = indices[title]
    
    # Get the pairwsie similarity scores of all books with that book
    sim_scores = list(enumerate(Desc_sim[idx]))

    # Sort the books based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar books
    sim_scores = sim_scores[1:10]

    # Get the book indices
    book_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar books
    return df['title'].iloc[book_indices]

In [35]:
book_recomendation('Golden Cup')

1446                                        Golden Orange
588             A Cup of Tea (Ballantine Reader's Circle)
2082    Dragonwings : Golden Mountain Chronicles: 1903...
224       The Golden Compass (His Dark Materials, Book 1)
792           City of Golden Shadow (Otherland, Volume 1)
3445    The Illuminatus Trilogy: The Eye in the Pyrami...
932     The Golden Mean: In Which the Extraordinary Co...
2916    The End Of The Dream The Golden Boy Who Never ...
0                                  The Kitchen God's Wife
Name: title, dtype: object

In [36]:
book_recomendation('Golden Cup').tolist()

['Golden Orange',
 "A Cup of Tea (Ballantine Reader's Circle)",
 'Dragonwings : Golden Mountain Chronicles: 1903 (Golden Mountain Chronicles)',
 'The Golden Compass (His Dark Materials, Book 1)',
 'City of Golden Shadow (Otherland, Volume 1)',
 'The Illuminatus Trilogy: The Eye in the Pyramid, the Golden Apple & Leviathan',
 'The Golden Mean: In Which the Extraordinary Correspondence of Griffin & Sabine Concludes',
 'The End Of The Dream The Golden Boy Who Never Grew Up : Ann Rules Crime Files Volume 5',
 "The Kitchen God's Wife"]

### Interfaz




In [37]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output# Load Data

In [53]:
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("Proyecto Final IA: Grupo 3"),
    html.H2("Sistema de recomendación de libros"),
    dcc.Input(
            id="book",
            type="text",
            placeholder="Input the book tittle",
            style={"width": "50%"},
        ),
    html.Br(),
    html.Br(),
    html.H3("KNN:"),
    html.Div(id="outputknn"),
    html.Br(),
    html.Br(),
    html.H3("TDFIF:"),
    html.Div(id="outputtdfif"),

])

@app.callback(
    Output("outputknn", "children"),
    Output("outputtdfif", "children"),
    Input("book", "value"),
)
def update_output(book):
    if book in listaDeTitulos:
        recknn = recoForInteface(book)
        rectdfif = book_recomendation(book).tolist()
        return u'books recommended for {}: {}'.format(recknn[0],recknn[1:]),u'books recommended for {}: {}'.format(book,rectdfif)
    else:
        rec = "book not found"
        return rec,rec


In [54]:
app.run_server(mode='external')

Dash app running on http://127.0.0.1:8050/


### Deploy